In [1]:
#import libraries
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# Final Presentation

## Introduction
For our project, we define "bestness" as safety. Thus, the safest neighborhood will be considered the best, while the most dangerous neighborhood to live in would be considered the worst. 


## Metric
There will exist $3$ submetrics within this project. Each submetric will produce a "ranking" of neighborhoods, as stored as a Series. Afterwards, the 



### Submetric 1: Police Incidents
Police Incidents will be the first sub-metric. We shall be using this dataset: https://data.wprdc.org/dataset/police-incident-blotter.
Each neighborhood will be assigned a "Bad Value," as determined by the following arbitrary equation: 

$N = z + \sum_{i_0}^{i_k}v_i$

Where $i$ is the $i$th incident of the neighborhood within the dataset, with $i_0$ being the first in the dataset and $i_k$ being the last,
where $v_i$ is the "bad" value of the $i$th incident(as determined by another ranking, based on the INCIDENTHIERARCHYDESC column, an **opinionated ranking**),
and where $z$ is the amount of incidents in that neighborhood.

Each neighborhood will be ranked, with the neighborhood with the lowest $N$ value being the best, while the neighborhood with the highest $N$ value being the worst.


In [ ]:
#Code, work, and more Markdown Above Submetric 2

### Submetric 2: Firearm Seizures

In [ ]:
#Code, work, and more Markdown Above Submetric 3

### Submetric 3: Police Arrests

In [ ]:
#Code, work, and more Markdown

## Conclusion

### Yanjia: Police Reported Incidents


### Parth:


### Adeel:


It has thus been decided that **XXXXXX** is the best neighborhood in Pittsburgh.